In [2]:
import numpy as np

In [127]:
def get_triplet_constraints(n, d):
    '''
    Generate all triplets
    '''
    X = np.random.randn(n,d)
    constraints = []
    
    
    x = np.reshape(X, (n*d,1))
    
    for i in range(n):
        for j in range(n):
            for k in range(j):
                if i!=j and i!=k and j!=k:
                    q = [i , j, k]
                    M = np.zeros((n*d,n*d))
                    if np.linalg.norm(X[q[0],:] - X[q[1],:]) < np.linalg.norm(X[q[0],:] - X[q[2],:]):
                        close = q[1]*d
                        far = q[2]*d                      
                    else: 
                        close = q[2]*d
                        far = q[1]*d
                        
                    for D in range(d):
                        M[close+D, close+D] = -1
                        M[far+D, far+D] = 1
                        
                        M[close+D, q[0]*d+D ]= 1
                        M[far + D, q[0]*d+D] = -1
                        M[q[0]*d+D, close+ D ]= 1
                        M[q[0]*d+D, far + D] = -1 
                    constraints.append(M)

    return constraints, x

In [128]:
def generate_trace_zero_constraints(dim, num_constraints):
    constraints = []
    for _ in range(num_constraints):
        M = np.random.randn(dim,dim)
        M = np.matmul(np.transpose(M),M)
        for i in range(dim):
            M[i,i] = 0
        i = np.random.randint(0,dim)
        M[i,i]=1
        j = np.random.randint(0,dim)
        while(i==j):
            j = np.random.randint(0,dim)
        M[j,j]=-1
        constraints.append(M)
    x = np.random.randn(dim,1)

    for i in range(len(constraints)):
        if np.matmul(np.transpose(x),np.matmul(constraints[i],x)) <0:
            constraints[i] = -1*constraints[i]
        elif np.matmul(np.transpose(x),np.matmul(constraints[i],x)) ==0:
            print('Oh no! Something evaluates to 0!')

    return constraints, x

In [98]:
def get_sigmoid_denom(x):
    return 1+np.exp(x)

In [165]:
def get_hessian_gradient_objective(dim, num_constraints, scale, true_test=False):
    hessian = np.zeros((dim,dim))
    grad = np.zeros((dim,1))
    obj = 0
    
    constraints,x = generate_trace_zero_constraints(dim, num_constraints)
    if true_test == True:
        x = scale*x
    else:
        x = np.random.randn(dim,1)
    for constraint in constraints:
        quad_form = np.matmul( np.matmul(np.transpose(x), constraint), x)
        denom = get_sigmoid_denom(quad_form)
        P_x = np.matmul(constraint, x)
        
        obj +=np.log(1+np.exp(-quad_form))
        grad += -2/denom * P_x
        hessian += 4*np.exp(quad_form) / (denom*denom)* np.matmul(P_x, np.transpose(P_x)) - 2/denom*constraint
    return hessian, grad, obj

In [166]:
def get_hessian_gradient_objective_triplet(dim, points,scale,true_test=False):
    hessian = np.zeros((dim*points,dim*points))
    grad = np.zeros((dim*points,1))
    obj = 0
    
    constraints,x = get_triplet_constraints(points, dim)
    if true_test == True:
        x = scale*x
    else:
        x = np.random.randn(dim*points,1)
    for constraint in constraints:
        quad_form = np.matmul( np.matmul(np.transpose(x), constraint), x)
        denom = get_sigmoid_denom(quad_form)
        P_x = np.matmul(constraint, x)
        
        obj +=np.log(1+np.exp(-quad_form))
        grad += -2/denom * P_x
        hessian += 4*np.exp(quad_form) / (denom*denom)* np.matmul(P_x, np.transpose(P_x)) - 2/denom*constraint
    return hessian, grad, obj

In [106]:
def check_eigvalues(x):
    flag_pos = 0
    flag_neg = 0
    category = ''
    for i in range(len(x)):
        if x[i]>0:
            flag_pos = 1
        elif x[i]<0:
            flag_neg = 1
    if flag_pos ==1 and flag_neg ==1:
        category = 'indefinite'
    elif flag_neg ==0 and flag_pos == 1:
        category = 'PSD'
    else:
        category = 'NSD'
    print(category)
    return category

In [173]:
trials = 100
psd = 0
nsd = 0
indef = 0
for _ in range(trials):
    #hessian, grad, obj = get_hessian_gradient_objective_triplet(2, 10, False,scale)
    hessian, grad, obj = get_hessian_gradient_objective(5, 100, False, 20)    
    print('gradient',np.linalg.norm(grad), 'objective',obj)
    eig_val = np.linalg.eig(hessian)[0]
    print(eig_val)
    category = check_eigvalues(eig_val)
    if category == 'indefinite':
        indef +=1
    elif category =='PSD':
        psd +=1
    else:
        nsd +=1
            
print('proportion indef', indef / float(trials))
print('proportion psd', psd / float(trials))
print('proportion nsd', nsd / float(trials))

gradient 287.441667648 objective [[ 225.95392765]]
[ 465.06039364  -49.31893619  260.0282928   136.54523635   66.40139505]
indefinite
gradient 627.555172485 objective [[ 665.44844387]]
[ 589.92288982 -184.55656327    6.87297354  273.1724519   239.60432418]
indefinite
gradient 896.30256937 objective [[ 1322.14715571]]
[-309.18356408  267.83984386  123.47680003   -4.59880601   50.39286495]
indefinite
gradient 227.744159463 objective [[ 108.50291445]]
[ 685.49052489  422.89410901    2.37137906  207.42344114   88.62512838]
PSD
gradient 241.987872397 objective [[ 146.10104968]]
[-134.50903475  -12.12429161  440.42273922  293.35032603  211.8460446 ]
indefinite
gradient 387.254465059 objective [[ 354.32523391]]
[ 449.36333362  -63.99443895   71.85367221  172.88117639  127.73552768]
indefinite
gradient 314.1127131 objective [[ 210.92962851]]
[-159.36867989  101.42750154  409.26024855  272.77023162  344.78983651]
indefinite
gradient 415.826229706 objective [[ 322.65797768]]
[-144.83241521  -16.

gradient 361.554838362 objective [[ 222.38164459]]
[ 1106.66747859   413.41674745   237.20909989   121.99407694   -18.46369916]
indefinite
gradient 607.01057456 objective [[ 841.65289218]]
[-141.04755962  472.42214861  252.08549053   84.64869924  176.16275065]
indefinite
gradient 367.626142376 objective [[ 277.21197793]]
[-159.22080483  479.00864307  300.04971713   58.86749916  171.14001714]
indefinite
gradient 199.834046908 objective [[ 105.8547229]]
[ -25.89962199   43.18650944  180.84075592  413.75211664  554.97770175]
indefinite
gradient 285.080101311 objective [[ 197.77733534]]
[   7.98390553   60.37847209  504.37115019  380.38097225  228.02398804]
PSD
gradient 471.764307523 objective [[ 387.65517888]]
[ 494.0965449   273.52422156  -48.67768028  121.53435092   20.94945539]
indefinite
gradient 417.956030273 objective [[ 366.59989599]]
[ -83.04871356   18.06440698  200.21518561  526.00277073  434.69660483]
indefinite
gradient 606.646493123 objective [[ 500.05581217]]
[ 693.02848879 

In [38]:
X = np.random.randn(3,2)

In [39]:
print(X)

[[-0.23182879  1.16931282]
 [ 0.66772429 -0.47582168]
 [ 0.54905949 -0.22580684]]


In [56]:
test= np.reshape(X, (6,1))

In [58]:
print(test)

[[-0.23182879]
 [ 1.16931282]
 [ 0.66772429]
 [-0.47582168]
 [ 0.54905949]
 [-0.22580684]]
